In [1]:
import numpy as np
import xarray as xr
import scipy.io as sio
import pandas as pd
import calendar
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import cartopy
import cartopy.crs as ccrs

# for shapefile
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature

# NCL colormap
import matplotlib
import colormath, colormath.color_objects, colormath.color_conversions
from colormath.color_objects import sRGBColor
import urllib
import re

In [3]:
rootdir = '/raid1/chen423/serdp/archive/GRL2020/'

plotdir = rootdir + 'plots/'

In [4]:
color_obj_dict = {'sRGB':colormath.color_objects.sRGBColor,
                  'HSV':colormath.color_objects.HSVColor,
                  'Lab':colormath.color_objects.LabColor,
                  'LCHuv':colormath.color_objects.LCHuvColor,
                  'LCHab':colormath.color_objects.LCHabColor,
                  'XYZ':colormath.color_objects.XYZColor}

def __rgb_to_array(rgb_color):
    r = np.minimum(1, round(rgb_color.rgb_r*10000)/10000)
    g = np.minimum(1, round(rgb_color.rgb_g*10000)/10000)
    b = np.minimum(1, round(rgb_color.rgb_b*10000)/10000)
    return r,g,b


def create_palette(start_rgb, end_rgb, n, colorspace):
    # convert start and end to a point in the given colorspace
    start = colormath.color_conversions.convert_color(start_rgb, colorspace).get_value_tuple()
    end = colormath.color_conversions.convert_color(end_rgb, colorspace).get_value_tuple()

    # create a set of n points along start to end
    points = list(zip(*[np.linspace(start[i], end[i], n) for i in range(3)]))

    # create a color for each point and convert back to rgb
    rgb_colors = [colormath.color_conversions.convert_color(colorspace(*point), sRGBColor) for point in points]

    # convert rgb colors to arrays
    return [__rgb_to_array(color) for color in  rgb_colors]


def __retrive_NCL_webcontent(cmapname):
    target_url = 'https://www.ncl.ucar.edu/Document/Graphics/ColorTables/Files/%s.rgb' % cmapname
    request = urllib.request.urlopen(target_url)
    return request


def __collect_discrete_NCL_cmap(cmapname):
    rawdata = __retrive_NCL_webcontent(cmapname)
    
    cmap_color_list = list()
    
    color_section_sig = 0
    
    for line in rawdata:
        
        line_decode = line.decode('utf-8')
        info = re.split('\s+', line_decode.replace('\n','').replace('^\s+',''))
        
        if color_section_sig==1:
            if info[0]=='' and len(info)>=3:
                if np.maximum(np.maximum(float(info[1]), float(info[2])), float(info[3]))>1:
                    cmap_color_list.append((float(info[1])/255, float(info[2])/255, float(info[3])/255))
                else:
                    cmap_color_list.append((float(info[1]), float(info[2]), float(info[3])))
            if len(info)==3:
                if ';' in info[0] or '#' in info[0]:
                    whatisthis = 's'
                else:
                    if np.maximum(np.maximum(float(info[0]), float(info[1])), float(info[2]))>1:
                        cmap_color_list.append((float(info[0])/255, float(info[1])/255, float(info[2])/255))
                    else:
                        cmap_color_list.append((float(info[0]), float(info[1]), float(info[2])))
        
        if 'ncolors' in str(info[0]):
            color_section_sig = 1  # meaning now we are at color lines (or "r g b" line)

    return cmap_color_list


def __cmap_refinement(raw_cmap_rgb, n_interpolate=10, workspace=color_obj_dict['sRGB']):
    # workspace:  choose which color space the refinement is conducted.
    #             refer to https://stackoverflow.com/questions/55032648/given-a-start-color-and-a-middle-color-how-to-get-the-remaining-colors-python
    
    n_in = len(raw_cmap_rgb)

    new_array = list()

    for i in np.arange(n_in-1):
        out_colors = create_palette(sRGBColor(*raw_cmap_rgb[i], is_upscaled=False), sRGBColor(*raw_cmap_rgb[i+1], is_upscaled=False), n_interpolate+1, workspace)
        for j in np.arange(len(out_colors)-1):
            new_array.append(out_colors[j])
            
    return new_array


def generate_NCL_cmap(cmapname, cont_opt=False, cont_param_n=10, cont_param_ws='sRGB',
                      white_first=False, white_ext=False, reverse_cmap=False):
    # description:
    #     cmapname:      taken as shown on the NCL website
    #     cont_opt:      to convert the discreate colormap to continuous colormap
    #     cont_param_n:  how many "intermediate" colors to be inserted to the nearby discreate colors
    #     cont_param_ws: color space to conduct interploation. Default to "sRGB", which should work for most cases
    #     white_first:   whether to set the first color as white. May be useful if the minimum does not mean anything
    
    cmap_discrete_raw = __collect_discrete_NCL_cmap(cmapname)
    
    if reverse_cmap==True:
        cmap_discrete_raw.reverse()
    
    if white_first==True:
        if white_ext==True:
            cmap_discrete = list()
            cmap_discrete.append((1,1,1))
            for i in np.arange(len(cmap_discrete_raw)):
                cmap_discrete.append(cmap_discrete_raw[int(i)])
        else:
            cmap_discrete = cmap_discrete_raw.copy()
        cmap_discrete[0] = (1,1,1)
    else:
        cmap_discrete = cmap_discrete_raw
    
    if cont_opt==False:
        out_cmap = cmap_discrete
        
    if cont_opt==True:
        out_cmap = __cmap_refinement(cmap_discrete, n_interpolate=cont_param_n, workspace=color_obj_dict[cont_param_ws])
        
    return matplotlib.colors.ListedColormap(out_cmap)

def crt_cbar_labels(vmax, n_interval, mode='diff', decimal_flag=0, perc_flag=False, vmin=0):
    # crt_cbar_labels:  create the colorbar label lists
    #   mode:  choose between "diff" and "0ton". "diff" means setting the colorbar as -vmax to vmax, "0ton" 
    #          means setting the colorbar as 0 to vmax
    #   n_interval:  how many segments are there? See example below.
    #   decimal_flag:  control the text format. Default to 0.
    #
    # Example:
    #      > crt_cbar_labels(80, 4, mode='diff', decimal_flag=0)
    #      >  ['-80', '-40', '0', '40', '80']
    #      > crt_cbar_labels(80, 4, mode='0ton', decimal_flag=1)
    #      >  ['0.0', '20.0', '40.0', '60.0', '80.0']
    
    if perc_flag==True:
        format_string = '%%.%df%%%%' % (decimal_flag)
    else:
        format_string = '%%.%df' % (decimal_flag)
    #print(format_string)
    outdata = []
    
    if mode=='diff':
        n_interval = n_interval/2
        for i in np.arange(-1*n_interval, n_interval+0.000001, 1):
            outdata.append(format_string%(vmax*i/n_interval))
    if mode=='0ton':
        for i in np.arange(0, n_interval+0.000001, 1):
            outdata.append(format_string%(vmax*i/n_interval))
    if mode=='minmax':
        for i in np.arange(0, n_interval+0.000001, 1):
            outdata.append(format_string%(vmin + (vmax-vmin)*i/n_interval))
        
    return outdata

In [5]:
def visualize_wUS_map(axis, lons, lats, indata, cmap='bwr_r', label='', color=False,
                      location=[False,False,False,False], norm=False, vmin=0, vmax=1, title='',
                      xlim=[-127,-100], ylim=[26,55], map_bdy=False, wUS_bdy=False, wUS_range=4, **kwarg):
    if color!=False:
        axis.pcolormesh(lons, lats, indata, color=color, **kwarg)
    else:
        if norm==False:
            axis.pcolormesh(lons, lats, indata, cmap=cmap, vmin=vmin, vmax=vmax, zorder=2, **kwarg)
        else:
            axis.pcolormesh(lons, lats, indata, cmap=cmap, norm=norm, zorder=2, **kwarg)

    axis.set_xlim(xlim)
    axis.set_ylim(ylim)
    
    if map_bdy==False:
        # turn off the bounding box for cartopy
        axis.outline_patch.set_visible(False)

    axis.add_feature(cartopy.feature.OCEAN, linewidth=0.5, facecolor='none', edgecolor='k', zorder=0)
    axis.add_feature(cartopy.feature.LAND, linewidth=0.5, facecolor='none', edgecolor='k', zorder=1)

    if wUS_bdy==True:
        shpfile = rootdir + 'data/common_ref/wUS_%dstates/US_states.shp' % (wUS_range)
        shape_feature = ShapelyFeature(Reader(shpfile).geometries(), ccrs.PlateCarree(), 
                                       facecolor='none', edgecolor='black', linewidth=1)
        axis.add_feature(shape_feature, zorder=4)

    countries = cartopy.feature.NaturalEarthFeature(category='cultural', scale='10m', edgecolor='grey', linewidth=0.5,\
                                                    facecolor='none', name='admin_1_states_provinces')
    axis.add_feature(countries, zorder=3)
    
    gl = axis.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linestyle='--', alpha=1)
    gl.xlabels_top = location[0]
    gl.xlabels_bottom = location[1]
    gl.ylabels_left = location[2]
    gl.ylabels_right = location[3]
    gl.xlocator = matplotlib.ticker.FixedLocator(np.arange(-180,-59,10))
    gl.ylocator = matplotlib.ticker.FixedLocator(np.arange(0,81,10))
    
    axis.set_title(title, size=12)
    axis.text(-124, 34, label, ha='left', va='top', size=12)

In [6]:
def plot_WRF_domain_boundary(axes, **kwargs):
    
    nx, ny = wrf_lats.shape[0:2]
    axes.plot(wrf_lons[0,:], wrf_lats[0,:], **kwargs)
    axes.plot(wrf_lons[nx-1,:], wrf_lats[nx-1,:], **kwargs)
    axes.plot(wrf_lons[:,0], wrf_lats[:,0], **kwargs)
    axes.plot(wrf_lons[:,ny-1], wrf_lats[:,ny-1], **kwargs)

In [7]:
def crt_filenames(model, year, month):
    
    WRFdir = rootdir + 'data/%s/WRF_IWV_uvIVT/' % model
    WRFfile = WRFdir + 'WRF_IWV_uvIVT.6hr.%d.%d.nc' % (year, month)
    
    return WRFfile

In [8]:
def get_AR_intensity_data(model, year, month):
    
    WRFfile = crt_filenames(model, year, month)
    
    WRF_IVT = xr.open_dataset(WRFfile).uvIVT.values
    WRF_IWV = xr.open_dataset(WRFfile).IWV.values
    
    return WRF_IVT, WRF_IWV

## 1. Find appropriate month

In [9]:
def crt_refdata():
    reffile = rootdir + 'data/common_ref/latlon.nc'
    lats = xr.open_dataset(reffile).XLAT_M.values
    lons = xr.open_dataset(reffile).XLONG_M.values
    
    return lats, lons
    
wrf_lats, wrf_lons = crt_refdata()

reffile = rootdir + 'data/common_ref/SERDP6km.dist_to_coastal.nc'
dist_to_coast = xr.open_dataset(reffile).dist_to_coast.values
dist_to_coast[dist_to_coast==9999] = 0
ocean_mask = np.zeros((450,450))
ocean_mask[dist_to_coast==0] = 1


reffile = rootdir + 'data/common_ref/US_state.nc'
USstate = 1-xr.open_dataset(reffile).state_mask.values[0:5].sum(axis=0)

In [10]:
ARtag = 'p85'

flag_area = 1000   # minimum size of patches
flag_USstate = 1  # whether to use US west coast 5 states along with land mask. 1 is to use, 0 is to skip
flag_post_adj = 1 # WRF further adjusted, or not (i.e., directly from modified NARR). 1 is further adjusted, 0 for raw

commonAR_thre = 1000

version_tag = 'AR%s_s%d_state%d_post%d_c%d' % (ARtag, flag_area, flag_USstate, flag_post_adj, commonAR_thre)
print(version_tag)

ARp85_s1000_state1_post1_c1000


In [11]:
year = 2005
month = 3

In [12]:
file_ARHIST = rootdir + 'data/HIST/AR_tag/Gershunov/SERDP6km_adj/WRF_ARtag_adj.HIST.Gershunov.%d.%d.AR%s.nc' % (year, month, ARtag)
file_ARfSST = rootdir + 'data/fSST/AR_tag/Gershunov/SERDP6km_adj/WRF_ARtag_adj.fSST.Gershunov.%d.%d.AR%s.nc' % (year, month, ARtag)

ARtag_HIST = xr.open_dataset(file_ARHIST).AR_tag.values
ARtag_fSST = xr.open_dataset(file_ARfSST).AR_tag.values

In [13]:
file_SSTHIST = rootdir + 'data/HIST/SST/NARR_TS.SERDP6km.6hourly.%d.%d.nc' % (year, month)
    
file_SSTfix = rootdir + 'data/HIST/SST/NARR_TS.SERDP6km.2000.10.01.00.nc'
    
    
SST_HIST = xr.open_dataset(file_SSTHIST).var11.values

SST_fSST = xr.open_dataset(file_SSTfix).var11.values[0]

In [14]:
IVT_HIST, IWV_HIST = get_AR_intensity_data('HIST', year, month)

IVT_fSST, IWV_fSST = get_AR_intensity_data('fSST', year, month)

FileNotFoundError: [Errno 2] No such file or directory: b'/raid1/chen423/serdp/archive/GRL2020/data/HIST/WRF_IWV_uvIVT/WRF_IWV_uvIVT.6hr.2005.3.nc'

## 2. SST data in panel a

In [ ]:
infile = rootdir + 'data/HIST/SST_background/NARR_TS.SERDP6km.6hourly.monsum.200310-201512.nc'
in_SSTmonmean_full = xr.open_dataset(infile).var11.values[0:144]

ts_monthly = pd.period_range(start='2003-10', end='2015-09', freq='M')

infile = rootdir + 'data/HIST/SST_background/NARR_TS.SERDP6km.2000.10.01.00.nc'
fSST_full = xr.open_dataset(infile).var11.values[0]

In [ ]:
def compute_SSTmean(in_SST):
    mask_full = ocean_mask==1
    out_SSTmean_all = in_SST[mask_full].mean()
    
    return out_SSTmean_all

In [ ]:
fSST = compute_SSTmean(fSST_full)

In [ ]:
SSTmean_mseries = np.zeros(144)
for i in np.arange(144):
    SSTmean_mseries[i] = compute_SSTmean(in_SSTmonmean_full[i])

In [ ]:
SSTmean_mcycle = np.zeros((12,3)) # month, region, [stats]

for i in np.arange(12):
    valid_index = ts_monthly.month==(i+1)
    SSTmean_mcycle[i,0] = SSTmean_mseries[valid_index].mean()+SSTmean_mseries[valid_index].std()/2
    SSTmean_mcycle[i,1] = SSTmean_mseries[valid_index].mean()
    SSTmean_mcycle[i,2] = SSTmean_mseries[valid_index].mean()-SSTmean_mseries[valid_index].std()/2

### major AR-related functions

In [ ]:
cmap_IVT = generate_NCL_cmap('precip3_16lev', cont_opt=False)
#cmap_IVT = matplotlib.cm.PuBuGn
#cmap_SST = generate_NCL_cmap('t2m_29lev', cont_opt=True)
#cmap_SST = matplotlib.cm.hot
cmap_SST = matplotlib.cm.pink

In [ ]:
t = 108   # or 93, for 2003-12 month

fig1 = plt.figure(figsize=(7,8))
ax1 = plt.subplot2grid((30,2), (9,0), rowspan=20, projection=ccrs.PlateCarree())
ax2 = plt.subplot2grid((30,2), (9,1), rowspan=20, projection=ccrs.PlateCarree())
ax3 = plt.subplot2grid((30,2), (0,0), rowspan=8, colspan=2)

visualize_wUS_map(ax1, wrf_lons, wrf_lats, np.ma.masked_array(IVT_HIST[t], mask=ARtag_HIST[t]==0),
                  cmap=cmap_IVT, vmin=200, vmax=700,
                  xlim=[-135,-113], ylim=[28,55], wUS_bdy=True)
visualize_wUS_map(ax2, wrf_lons, wrf_lats, np.ma.masked_array(IVT_fSST[t], mask=ARtag_HIST[t]==0),
                  cmap=cmap_IVT, vmin=200, vmax=700,
                  xlim=[-135,-113], ylim=[28,55], wUS_bdy=True)

ax1.pcolormesh(wrf_lons, wrf_lats, np.ma.masked_array(SST_HIST[t], mask=ocean_mask==0),
               cmap=cmap_SST, vmin=280, vmax=295, alpha=0.5)
ax2.pcolormesh(wrf_lons, wrf_lats, np.ma.masked_array(SST_fSST, mask=ocean_mask==0),
               cmap=cmap_SST, vmin=280, vmax=295, alpha=0.5)

ax1.contour(wrf_lons, wrf_lats, ARtag_HIST[t], cmap='Greys', levels=[0,0.99,1], linewidths=1, zorder=10)
ax2.contour(wrf_lons, wrf_lats, ARtag_fSST[t], cmap='Greys', levels=[0,0.99,1], linewidths=1, zorder=10)

for axis in [ax1, ax2]:
    plot_WRF_domain_boundary(axis, color='k', linestyle='--')
    
ax3.plot(np.arange(1,13), SSTmean_mcycle[:,1], color='royalblue', label='CTRL (mean)')
ax3.fill_between(np.arange(1,13), SSTmean_mcycle[:,0], SSTmean_mcycle[:,2],
                 color='royalblue', alpha=0.4, label='CTRL (std)')
ax3.plot(np.arange(1,13), np.ones(12)*fSST, color='black', linestyle='-', linewidth=2, label='fSST')
ax3.text(12.2, fSST, '%.1f K'%fSST, ha='left', va='center', fontsize=10, rotation=90)
for t in np.arange(280, 292, 2):
    ax3.plot(np.arange(1,13), np.ones(12)*t, color='grey', linestyle='--', linewidth=1)

ax3.legend(loc=(0.55, 0.03), ncol=1, frameon=False, fontsize=12)
ax3.set_xlim([1,12])
ax3.set_ylim([284, 293])
ax3.set_xticks(np.arange(1, 13))
ax3.set_yticks(np.arange(284, 292.1, 2))
ax3.set_xlabel('month', fontsize=12)
ax3.set_ylabel('mean SST (K)', fontsize=12)
ax3.spines['right'].set_linewidth(1)
ax3.spines['top'].set_linewidth(1)


cbar_ax1 = fig1.add_axes([0.12, 0.12, 0.35, 0.015])
cb1 = matplotlib.colorbar.ColorbarBase(cbar_ax1, cmap=cmap_IVT, ticks=np.arange(0, 1.001, 0.25), orientation='horizontal')
cb1.set_ticklabels(crt_cbar_labels(700, 4, mode='minmax', vmin=200, decimal_flag=0))
cbar_ax1.tick_params(labelsize=10)
cbar_ax1.text(0.5, 1.5, r'IVT ($kg \cdot m^{-1}s^{-1}$)', ha='center', va='bottom', fontsize=12)

cbar_ax2 = fig1.add_axes([0.55, 0.12, 0.35, 0.015])
cb2 = matplotlib.colorbar.ColorbarBase(cbar_ax2, cmap=cmap_SST, ticks=np.arange(0, 1.001, 0.25), orientation='horizontal')
cb2.set_ticklabels(crt_cbar_labels(295, 4, mode='minmax', vmin=280, decimal_flag=0))
cbar_ax2.tick_params(labelsize=12)
cbar_ax2.text(0.5, 1.5, 'SST (K)', ha='center', va='bottom', fontsize=12)

ax1.text(-134, 29, '(b)\nCTRL', ha='left', va='bottom', fontsize=14, backgroundcolor='white')
ax2.text(-134, 29, '(c)\nfSST', ha='left', va='bottom', fontsize=14, backgroundcolor='white')
ax3.text(1.2, 292.6, '(a) SST monthly cycle', ha='left', va='top', fontsize=14, backgroundcolor='white')

#fig1.savefig(plotdir + 'fig01.schematic_newcolors.png', dpi=600)

plt.show()
plt.close()
del(fig1)